In [ ]:
#Tu jest syf

In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon
import plotly
import json
import plotly.express as px
import skmob
import datetime
from tqdm import tqdm
from itertools import islice

In [2]:
df = pd.read_csv("Wroclaw_detail.csv", encoding='latin1', low_memory=False)

In [45]:
df2 = pd.read_csv("Wroclaw_generic_c.csv", encoding='latin1', low_memory=False, error_bad_lines=False)

In [3]:
geometry = [Point(xy) for xy in zip(df[" Longitude"], df["Latitude"])]

In [4]:
geo_df = gpd.GeoDataFrame(df, crs = "EPSG:4326", geometry = geometry)
geo_df.head()

,TripID,TimeStamp,Latitude,Longitude,Altitude,Distance,Speed,Type,geometry
0,574f1cff88c537e377a3e136,1.462552e+09,51.09043,17.03320,157,0.000000,3.400368,start,POINT (17.03320 51.09043)
1,574f1cff88c537e377a3e136,1.462552e+09,51.09028,17.03304,156,0.020065,3.041381,mid,POINT (17.03304 51.09028)
2,574f1cff88c537e377a3e136,1.462552e+09,51.09017,17.03323,155,0.018037,3.783186,mid,POINT (17.03323 51.09017)
3,574f1cff88c537e377a3e136,1.462552e+09,51.08995,17.03309,159,0.026330,4.527693,mid,POINT (17.03309 51.08995)
4,574f1cff88c537e377a3e136,1.462552e+09,51.08979,17.03303,160,0.018268,4.776243,mid,POINT (17.03303 51.08979)


In [5]:
df['geometry'] = geo_df['geometry']

In [6]:
wroc_map = gpd.read_file('EtapII-REJONY_wroclaw.shp')
city_regions = wroc_map[['NUMBER', 'geometry']]
city_regions.columns = ['REGION', 'geometry']

In [7]:
city_regions = city_regions.to_crs(epsg=4326)

In [8]:
def point_in(regions, point):
    for index, row in regions.iterrows():
        if point.within(row['geometry']):
            return row['REGION']
    return 0  

In [19]:
def count_regions(df):
    df_counts = pd.DataFrame()
    df_counts['TripID'] = df['TripID'].unique()[4000:5000]
    print(len(df_counts['TripID']))
    for index, row in tqdm(df_counts.iterrows(), total=df_counts.shape[0]):
        df_this_trip = df[df['TripID']==row['TripID']]
        #print(len(df_this_trip['TripID']))
        regions = set()
        for index_trip, row_trip in islice(df_this_trip.iterrows(),0,len(df_this_trip['TripID']),5):
            regions.add(point_in(city_regions,row_trip['geometry']))
        #print(regions)
        #print(len(regions))
        df_counts.at[index,'count'] = len(regions)
    return df_counts

In [21]:
df_count = count_regions(df)

  0%|                                                                                         | 0/1000 [00:00<?, ?it/s]

1000


100%|████████████████████████████████████████████████████████████████████████████| 1000/1000 [1:33:56<00:00,  5.64s/it]


In [23]:
df_count.to_csv('region_num6.csv')

In [34]:
df_start = df[df[' Type']=='start']
#df_mid = df[df[' Type']=='mid']
df_end = df[df[' Type']=='end']

In [94]:
df_start = df_start[['geometry','TripID']]
df_end = df_end[['geometry','TripID']]
df_start_end = df_start.set_index('TripID').join(df_end.set_index('TripID'),lsuffix = '_start', rsuffix = '_end')
df_start_end

,geometry_start,geometry_end
TripID,,
574f1cff88c537e377a3e136,POINT (17.03320 51.09043),POINT (16.97578 51.13789)
574e425b88c537566aa3e0fe,POINT (16.99408 51.11318),POINT (17.12595 51.14234)
574e089988c537295ba3e0fa,POINT (17.07723 51.10846),POINT (17.10334 51.11365)
574e0a1388c5376877a3e0f6,POINT (17.07771 51.10812),POINT (17.03058 51.09683)
574e076e88c537486aa3e0f8,POINT (17.00275 51.10800),POINT (16.95687 51.11799)
...,...,...
5732351d88c5377b537b23c7,POINT (16.92915 51.10514),POINT (16.97066 51.13763)
5732351d88c537e2317b2422,POINT (16.98753 51.14551),POINT (16.92912 51.10509)
5571e7bd88c53764431b7d2a,POINT (16.94769 51.10020),POINT (16.95050 51.09971)


In [95]:
df_start_end = df_start_end.dropna()

In [96]:
df_start_end

,geometry_start,geometry_end
TripID,,
574f1cff88c537e377a3e136,POINT (17.03320 51.09043),POINT (16.97578 51.13789)
574e425b88c537566aa3e0fe,POINT (16.99408 51.11318),POINT (17.12595 51.14234)
574e089988c537295ba3e0fa,POINT (17.07723 51.10846),POINT (17.10334 51.11365)
574e0a1388c5376877a3e0f6,POINT (17.07771 51.10812),POINT (17.03058 51.09683)
574e076e88c537486aa3e0f8,POINT (17.00275 51.10800),POINT (16.95687 51.11799)
...,...,...
5732351d88c5377b537b23c7,POINT (16.92915 51.10514),POINT (16.97066 51.13763)
5732351d88c537e2317b2422,POINT (16.98753 51.14551),POINT (16.92912 51.10509)
5571e7bd88c53764431b7d2a,POINT (16.94769 51.10020),POINT (16.95050 51.09971)


In [48]:
travels = pd.read_csv("travels.csv",usecols=['ORIGIN','DESTINATION'])

In [49]:
travels

,ORIGIN,DESTINATION
0,63,149
1,119,299
2,73,81
3,73,22
4,52,163
...,...,...
42130,253,149
42131,112,253
42132,347,347
42133,171,347


In [50]:
travels2 = travels.groupby(['ORIGIN','DESTINATION']).size().reset_index(name='FLOW')

In [52]:
travels2 = travels2[travels2['ORIGIN']!=0]
travels2 = travels2[travels2['DESTINATION']!=0]

In [54]:
travels2.reset_index()

,index,ORIGIN,DESTINATION,FLOW
0,218,1,1,24
1,219,1,2,4
2,220,1,3,3
3,221,1,4,3
4,222,1,5,3
...,...,...,...,...
13710,14162,454,140,5
13711,14163,454,313,1
13712,14164,454,342,2
13713,14165,454,350,1


In [56]:
travels2.to_csv("flows.csv")

In [57]:
travels_from_count = travels.groupby(['ORIGIN']).size().reset_index(name='FLOW')

In [59]:
travels_from_count = travels_from_count[travels_from_count['ORIGIN']!=0]

In [62]:
travels_from_count

,ORIGIN,FLOW
1,1,530
2,2,277
3,3,259
4,4,318
5,5,244
...,...,...
362,401,45
363,451,44
364,452,17
365,453,37


In [96]:
start_region = {}
for i in range(1,455):
    if i in travels_from_count.ORIGIN.values:
        flow_num = travels_from_count['FLOW'].loc[travels_from_count['ORIGIN'] == i].values[0]
        start_region[i] = str(flow_num)
    else:
        start_region[i] = '0'

In [97]:
print(start_region)

{1: '530', 2: '277', 3: '259', 4: '318', 5: '244', 6: '217', 7: '57', 8: '75', 9: '68', 10: '212', 11: '94', 12: '56', 13: '377', 14: '252', 15: '145', 16: '213', 17: '108', 18: '141', 19: '94', 20: '140', 21: '110', 22: '165', 23: '460', 24: '55', 25: '60', 26: '177', 27: '318', 28: '224', 29: '179', 30: '303', 31: '84', 32: '156', 33: '181', 34: '169', 35: '302', 36: '173', 37: '59', 38: '139', 39: '314', 40: '120', 41: '17', 42: '83', 43: '9', 44: '137', 45: '28', 46: '157', 47: '213', 48: '318', 49: '180', 50: '498', 51: '2', 52: '81', 53: '99', 54: '355', 55: '166', 56: '71', 57: '305', 58: '70', 59: '252', 60: '143', 61: '145', 62: '110', 63: '95', 64: '54', 65: '333', 66: '273', 67: '222', 68: '222', 69: '170', 70: '77', 71: '317', 72: '93', 73: '234', 74: '47', 75: '18', 76: '75', 77: '78', 78: '144', 79: '112', 80: '158', 81: '169', 82: '10', 83: '60', 84: '163', 85: '52', 86: '62', 87: '24', 88: '5', 89: '37', 90: '63', 91: '316', 92: '40', 93: '149', 94: '158', 95: '75', 96:

In [98]:
with open('start_region_dist.json', 'w') as outfile:
    json.dump(start_region, outfile)

In [89]:
list_files = []
list_files.append(pd.read_csv('region_num1.csv'))
list_files.append(pd.read_csv('region_num2.csv'))
list_files.append(pd.read_csv('region_num3.csv'))
list_files.append(pd.read_csv('region_num4.csv'))
list_files.append(pd.read_csv('region_num5.csv'))
list_files.append(pd.read_csv('region_num6.csv'))
list_files.append(pd.read_csv('region_num7.csv'))
list_files.append(pd.read_csv('region_num8.csv'))
list_files.append(pd.read_csv('region_num9.csv'))
region_num = pd.concat(list_files, axis=0, ignore_index=True)

In [90]:
max(region_num['count'])

136.0

In [91]:
region_num = region_num.rename(columns={'count':'trip_len_regions'})

In [92]:
region_num = region_num[['TripID','trip_len_regions']]

In [93]:
region_num


,TripID,trip_len_regions
0,574f1cff88c537e377a3e136,24.0
1,574e425b88c537566aa3e0fe,27.0
2,574e089988c537295ba3e0fa,4.0
3,574e0a1388c5376877a3e0f6,10.0
4,574e076e88c537486aa3e0f8,7.0
...,...,...
9995,574364cc88c537f86d92ee7a,23.0
9996,5743495e88c5373e3692ee77,25.0
9997,57434af888c537031192ee4d,7.0
9998,57433b9c88c537601a92eead,4.0


In [97]:
df_start_end = df_start_end.join(region_num.set_index('TripID'))

In [98]:
df_start_end

,geometry_start,geometry_end,trip_len_regions
TripID,,,
574f1cff88c537e377a3e136,POINT (17.03320 51.09043),POINT (16.97578 51.13789),24.0
574e425b88c537566aa3e0fe,POINT (16.99408 51.11318),POINT (17.12595 51.14234),27.0
574e089988c537295ba3e0fa,POINT (17.07723 51.10846),POINT (17.10334 51.11365),4.0
574e0a1388c5376877a3e0f6,POINT (17.07771 51.10812),POINT (17.03058 51.09683),10.0
574e076e88c537486aa3e0f8,POINT (17.00275 51.10800),POINT (16.95687 51.11799),7.0
...,...,...,...
5732351d88c5377b537b23c7,POINT (16.92915 51.10514),POINT (16.97066 51.13763),NaN
5732351d88c537e2317b2422,POINT (16.98753 51.14551),POINT (16.92912 51.10509),NaN
5571e7bd88c53764431b7d2a,POINT (16.94769 51.10020),POINT (16.95050 51.09971),NaN


In [99]:
df2 = df2[['TripID', ' TipeOfTrip']]

In [100]:
df_start_end = df_start_end.join(df2.set_index('TripID'))

In [101]:
df_start_end

,geometry_start,geometry_end,trip_len_regions,TipeOfTrip
TripID,,,,
5571e7bd88c53764431b7d2a,POINT (16.94769 51.10020),POINT (16.95050 51.09971),NaN,--
5571e7c488c53765431b7d2a,POINT (16.94710 51.10641),POINT (16.95052 51.09970),NaN,--
5571e7d088c537e8411b7d2a,POINT (16.95034 51.09967),POINT (16.95053 51.09972),NaN,--
5571e7d088c537e8411b7d2a,POINT (16.95034 51.09967),POINT (16.95053 51.09972),NaN,--
568973f388c537bb0ea7a298,POINT (16.59866 51.83677),POINT (16.58993 51.83796),NaN,--
...,...,...,...,...
574ee11c88c537d177a3e128,POINT (16.99491 51.08023),POINT (16.99381 51.07990),NaN,--
574ee14688c537b043a3e116,POINT (16.99491 51.08023),POINT (16.99381 51.07990),NaN,--
574ee15888c537712fa3e125,POINT (16.99491 51.08023),POINT (16.99381 51.07990),NaN,--


In [103]:
df_not_nan[' TipeOfTrip'].value_counts()

--              4981
HomeToWork      2442
Leisure         1532
Other            848
HomeToSchool     181
Name:  TipeOfTrip, dtype: int64

In [102]:
df_not_nan = df_start_end.dropna()

In [52]:
df_not_nan

,geometry_start,geometry_end,trip_len_regions,TipeOfTrip
TripID,,,,
5746cb0c88c537441292ef2d,POINT (16.98580 51.09153),POINT (17.00075 51.09926),4.0,Leisure
5746cbf688c537947192ef48,POINT (17.02618 51.14255),POINT (17.03012 51.10987),12.0,Leisure
5746ce0088c537596792eefa,POINT (17.05020 51.11848),POINT (17.08846 51.12540),8.0,Leisure
5746ce5f88c5377b6192ef42,POINT (17.05654 51.11934),POINT (17.03511 51.11026),8.0,--
5746d0d488c537450992ef41,POINT (16.92987 51.18257),POINT (16.95091 51.16068),2.0,--
...,...,...,...,...
574eddfa88c537e377a3e127,POINT (16.97581 51.11276),POINT (17.05269 51.12702),12.0,--
574ede1788c537a96aa3e12b,POINT (17.05266 51.12706),POINT (16.97588 51.11277),9.0,--
574ede2f88c5376f2fa3e116,POINT (16.97581 51.11276),POINT (17.05269 51.12702),12.0,--


In [104]:
for index, row in df_not_nan.iterrows():
    df_not_nan.at[index,'start'] = point_in(city_regions, row['geometry_start'])

C:\Users\User\Anaconda3\lib\site-packages\pandas\core\indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\User\Anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [105]:
df_not_nan

,geometry_start,geometry_end,trip_len_regions,TipeOfTrip,start
TripID,,,,,
57431df288c5374c2592ee87,POINT (17.04874 51.10456),POINT (17.07331 51.11798),7.0,--,18.0
5743262b88c537497292ee89,POINT (17.07289 51.11807),POINT (17.05859 51.13599),5.0,--,454.0
57432ad788c537346c92ee38,POINT (17.05801 51.13616),POINT (17.05041 51.13543),1.0,--,94.0
574333b788c537516c92eebb,POINT (17.12221 51.14100),POINT (17.12594 51.13655),1.0,Leisure,297.0
5743341a88c5375f1a92eeab,POINT (16.95715 51.09841),POINT (16.97521 51.09266),3.0,--,175.0
...,...,...,...,...,...
574eddfa88c537e377a3e127,POINT (16.97581 51.11276),POINT (17.05269 51.12702),12.0,--,125.0
574ede1788c537a96aa3e12b,POINT (17.05266 51.12706),POINT (16.97588 51.11277),9.0,--,34.0
574ede2f88c5376f2fa3e116,POINT (16.97581 51.11276),POINT (17.05269 51.12702),12.0,--,125.0


In [106]:
import random
for index, row in df_not_nan.iterrows():
    if row[' TipeOfTrip'] == '--' and random.randint(0, 10) <=3:
        df_not_nan.at[index,' TipeOfTrip'] = 'Leisure'

C:\Users\User\Anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [107]:
def generate_trip_len_dict(df_not_nan):
    trip_lenghts = {}
    for i in range(1,455):
        if i in df_not_nan.start.values:
            df_region = df_not_nan[df_not_nan['start']==i]
            region_trip_lenghts = {}
            for j in range(1,137):
                df_len = df_region[df_region['trip_len_regions']==j]
                region_trip_lenghts[j] = df_len.shape[0]
        else:
            for j in range(1,136):
                region_trip_lenghts[j] = '0'
        trip_lenghts[i] = region_trip_lenghts
    return trip_lenghts

In [108]:
df_work = df_not_nan[df_not_nan[' TipeOfTrip']!='Leisure']
df_leisure = df_not_nan[df_not_nan[' TipeOfTrip']=='Leisure']

In [109]:
json_work = generate_trip_len_dict(df_work)
json_leisure = generate_trip_len_dict(df_leisure)

In [86]:
print(json_leisure)

{1: {1: 0, 2: 1, 3: 1, 4: 0, 5: 1, 6: 0, 7: 2, 8: 3, 9: 0, 10: 4, 11: 0, 12: 0, 13: 3, 14: 5, 15: 1, 16: 1, 17: 0, 18: 1, 19: 0, 20: 2, 21: 0, 22: 1, 23: 1, 24: 0, 25: 0, 26: 2, 27: 0, 28: 0, 29: 1, 30: 0, 31: 0, 32: 0, 33: 0, 34: 0, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0, 41: 0, 42: 0, 43: 0, 44: 0, 45: 0, 46: 0, 47: 0, 48: 0, 49: 0, 50: 0, 51: 0, 52: 0, 53: 0, 54: 0, 55: 0, 56: 0, 57: 0, 58: 0, 59: 0, 60: 0, 61: 0, 62: 0, 63: 0, 64: 0, 65: 0, 66: 0, 67: 0, 68: 0, 69: 0, 70: 0, 71: 0, 72: 0, 73: 0, 74: 0, 75: 0, 76: 0, 77: 0, 78: 0, 79: 0, 80: 0, 81: 0, 82: 0, 83: 0, 84: 0, 85: 0, 86: 0, 87: 0, 88: 0, 89: 0, 90: 0, 91: 0, 92: 0, 93: 0, 94: 0, 95: 0, 96: 0, 97: 0, 98: 0, 99: 0, 100: 0, 101: 0, 102: 0, 103: 0, 104: 0, 105: 0, 106: 0, 107: 0, 108: 0, 109: 0, 110: 0, 111: 0, 112: 0, 113: 0, 114: 0, 115: 0, 116: 0, 117: 0, 118: 0, 119: 0, 120: 0, 121: 0, 122: 0, 123: 0, 124: 0, 125: 0, 126: 0, 127: 0, 128: 0, 129: 0, 130: 0, 131: 0, 132: 0, 133: 0, 134: 0, 135: 0, 136: 0}, 2: {1: 0, 2: 0

In [110]:
with open('trip_len_work.json', 'w') as outfile:
    json.dump(json_work, outfile)

In [111]:
with open('trip_len_leisure.json', 'w') as outfile:
    json.dump(json_leisure, outfile)